# Setup

In [ ]:
import pandas as pd
import numpy as np

import pandapower as pp
import pandapower.networks as pnet
import pandapower.plotting as pplot
import pandapower.timeseries as pts
import pandapower.control as pcon

from pandapower.plotting.plotly import pf_res_plotly
from pandapower.timeseries.output_writer import OutputWriter

# Network simulation
To start, we simulate Gaussian and independent fluctuations for power demands of loads, close to their referece values.

In [ ]:
net = pnet.case6ww()
samples = 100
load_sd_frac = 0.02

In [ ]:
n_load = net.load.shape[0]

In [ ]:
np.random.seed(11)
load_p = np.random.normal(net.load.p_mw, net.load.p_mw * load_sd_frac, (samples, n_load))
load_q = np.random.normal(net.load.q_mvar, net.load.q_mvar * load_sd_frac, (samples, n_load))

load_p_df = pts.DFData(pd.DataFrame(load_p))
const_load_p = pcon.ConstControl(net, element='load', element_index=net.load.index,
                                    variable='p_mw',  data_source=load_p_df, profile_name=net.load.index)
load_q_df = pts.DFData(pd.DataFrame(load_q))
const_load_q = pcon.ConstControl(net, element='load', element_index=net.load.index,
                                    variable='q_mvar',  data_source=load_q_df, profile_name=net.load.index)

In [ ]:
ow = OutputWriter(net, output_path='.', output_file_type=".csv", log_variables=[('res_bus', 'vm_pb')])
pts.run_timeseries(net, numba=False)

In [ ]:
net['_ppc']['internal']['Ybus'].todense()